In [1]:
import numpy as np
import pandas as pd
from keras.layers import Dense, Dropout, Activation, LSTM, Convolution1D, MaxPooling1D, Flatten
from keras.models import Sequential
import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'keras'

In [ ]:
## Data preparation

In [ ]:
data= pd.read_csv('^DJI.csv')
data.tail()

In [ ]:
data= data.dropna(axis=0)
data.tail()

In [ ]:
data= data.values[:, 1:5]
print (data.shape)

In [ ]:
print (data[:6, :])

In [ ]:
y= data[1:, 3]
print (y.shape)
X= data[:4283,:]
print (X.shape)

In [ ]:
# y= (y- min(y))/(max(y)- min(y))
print (y[4270:])
print (y[:7])

In [ ]:
# X= (X- np.min(X,axis=0))/(np.max(X, axis=0)-np.min(X,axis=0))
print (X.shape)
print (X[:3, :])

In [ ]:
## Input layer preparation

In [ ]:
time_steps= 100
X_new= np.zeros((X.shape[0] - time_steps +1, 100, X.shape[1]))
y_new= np.zeros((y.shape[0] -time_steps +1,))
for ix in range(X_new.shape[0]):
    for jx in range(time_steps):
        X_new[ix, jx, :]= X[ix +jx, :]
    y_new[ix]= y[ix + time_steps -1]
print (X_new.shape, y_new.shape)

In [ ]:
y_new

In [ ]:
## split into train and test sets

In [ ]:
split = int(0.8*data.shape[0])
X_train = X_new[:split]
X_test = X_new[split:]

y_train = y_new[:split]
y_test = y_new[split:]

print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

In [ ]:
# data normalisation after split the train/test

y_train= (y_train- min(y_train))/(max(y_train)- min(y_train))
X_train= (X_train- np.min(X_train,axis=0))/(np.max(X_train, axis=0)-np.min(X_train,axis=0))

y_test= (y_test- min(y_test))/(max(y_test)- min(y_test))
X_test= (X_test- np.min(X_test,axis=0))/(np.max(X_test, axis=0)-np.min(X_test,axis=0))

In [ ]:
## Model fitting

In [ ]:
model = Sequential()
model.add(Convolution1D(64, 3, input_shape= (100,4), padding='same'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))
model.add(Convolution1D(32, 3, padding='same'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=1))
model.add(Activation('linear'))

model.summary()

In [ ]:
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=256, epochs=50, validation_data = (X_test, y_test), verbose=2)

In [ ]:
score= model.evaluate(X_test, y_test, batch_size=256)

In [ ]:
print (score)

In [ ]:
y_pred= model.predict(X_test)
y_pred_train = model.predict(X_train)

In [ ]:
range(len(y_train),len(y_pred))

In [ ]:

plt.plot(range(len(y_train), len(y_train)+len(y_test)), y_test, 'b-')
# plt.plot(range(len(y_pred_train)), y_pred_train, 'r-')
# plt.plot(range(len(y_train)), y_train, 'b-')
plt.plot(range(len(y_train),len(y_train)+len(y_pred)), y_pred, 'r-')
plt.show()

In [ ]:
plt.plot(y_pred[400:600], 'r-')
plt.plot(y_test[400:600], 'b-')
plt.show()

In [ ]:
plt.plot(y_pred[400:450], 'r-')
plt.plot(y_test[400:450], 'b-')
plt.show()

In [ ]:
plt.plot (y_pred, y_test, 'bo')
plt.xlabel('y_pred')
plt.ylabel('y_test')
plt.show()